<a href="https://colab.research.google.com/github/rrm003/cc-lab04/blob/main/lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a64f6760e67b7fa30066174dcbbd783db609f29662f45f53ea10409b8e300ea3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

# Creating spark session
session = SparkSession.builder.master("local").appName("Test").getOrCreate()

In [ ]:
path = "./words.txt"
df = session.read.text(path)
df.show()

+---------+
|    value|
+---------+
|        A|
|        a|
|       aa|
|      aal|
|    aalii|
|      aam|
|     Aani|
| aardvark|
| aardwolf|
|    Aaron|
|  Aaronic|
|Aaronical|
| Aaronite|
|Aaronitic|
|     Aaru|
|       Ab|
|      aba|
|  Ababdeh|
|   Ababua|
|     abac|
+---------+
only showing top 20 rows



In [ ]:
# 1.1 List the 5 first words (in ascending order) and from the words.txt which start with “b” and end with “t”.

# filter data to get strings starting from b and endind with t
dfwithfilter = df.filter(df.value.like('b%t'))
dfwithfilter.show()

+----------+
|     value|
+----------+
|   babbitt|
|babblement|
|   babelet|
|baboonroot|
|    baboot|
|   babroot|
|  baccarat|
|  bacchant|
|  backcast|
|  backchat|
| backcourt|
|    backet|
| backjoint|
|   backlet|
|  backmost|
|   backset|
| backshift|
| backsight|
| backswept|
|  backwort|
+----------+
only showing top 20 rows



In [ ]:
# order in ascending order and get first 5.
dfwithfilter.orderBy(df.value).limit(5).show()

+----------+
|     value|
+----------+
|   babbitt|
|babblement|
|   babelet|
|baboonroot|
|    baboot|
+----------+



In [ ]:
# 1.2 List the 10 last longest words from the file words.txt.

from pyspark.sql.functions import col,length,trim

In [ ]:
# order the string data in descending order of lenght and filter top 10 records.
df.orderBy(length(col("value")).desc()).limit(10).show()

+--------------------+
|               value|
+--------------------+
|formaldehydesulph...|
|pathologicopsycho...|
|scientificophilos...|
|tetraiodophenolph...|
|thyroparathyroide...|
|anthropomorpholog...|
|hematospectrophot...|
|macracanthrorhync...|
|pericardiomediast...|
|pancreaticoduoden...|
+--------------------+



In [ ]:
# 1.3 Calculate the number of lines and the number of distinct words from file1.
f1path = './file1.txt'
dff1 = session.read.text(f1path)

#  number of lines
dff1.count()

372624

In [ ]:
# number of distinct words
dfDstinct = df.distinct()

In [ ]:
dfDstinct.show()

+--------------+
|         value|
+--------------+
|      abeltree|
|         Abner|
|   abnormalize|
|     abnormous|
|   Abrahamitic|
|    abruptness|
|     acanthoma|
|  Acanthopanax|
|   Acanthophis|
|      accensor|
|  accumulation|
|       acidity|
|        acnode|
|   acromegalia|
|   adaptionism|
|   additionary|
|      adjudger|
|      Aesopian|
|afflictionless|
|      affusion|
+--------------+
only showing top 20 rows



In [ ]:
dfDstinct.count()

235886

In [ ]:
# 1.4 Find 3 common words in files1, file2, and file3 which their sum of frequencies (number of occurrences)
# within the three files is maximum compared to the other shred words. (e.g., assume that X is a set of words which
# are shared between all files, find common words like xi in X which maximize
# F(xi)= f1(count xi in file 1) +f2(count xi in file 2) +f3(count xi in file 3).

In [ ]:
sc = session.sparkContext

In [ ]:
lines1 = sc.textFile("file1.txt")  # this is an RDD

test1 = lines1.zipWithIndex().flatMap(lambda x: ((i,(x[1],1)) for i in x[0].split(" ")))
test2 = test1.map(lambda x: ((x[0], x[1][0]), x[1][1])).reduceByKey(lambda x, y: x + y)
Result_RDD = test2.map(lambda x: (x[0][0], (x[0][1], x[1])))

In [ ]:
Result_RDD.take(4)

[('against', (302, 1)),
 ('of', (316, 1)),
 ('meaning', (459, 1)),
 ('afterwards', (648, 1))]

In [ ]:
lines2 = sc.textFile("file2.txt")
test1_2 = lines2.zipWithIndex().flatMap(lambda x: ((i,(x[1],1)) for i in x[0].split(" ")))
test2_2 = test1_2.map(lambda x: ((x[0], x[1][0]), x[1][1])).reduceByKey(lambda x, y: x + y)
Result_RDD2 = test2_2.map(lambda x: (x[0][0], (x[0][1], x[1])))

In [ ]:
lines3 = sc.textFile("file3.txt")
test1_3 = lines3.zipWithIndex().flatMap(lambda x: ((i,(x[1],1)) for i in x[0].split(" ")))
test2_3 = test1_3.map(lambda x: ((x[0], x[1][0]), x[1][1])).reduceByKey(lambda x, y: x + y)
Result_RDD3 = test2_3.map(lambda x: (x[0][0], (x[0][1], x[1])))

In [ ]:
Result_RDD = Result_RDD.union(Result_RDD2).union(Result_RDD3)

In [ ]:
Result_RDD = Result_RDD.union(Result_RDD2).union(Result_RDD3)

Result_RDD.take(4)

[('Vespignano,', (269, 1)),
 ('lay', (304, 1)),
 ('back', (583, 1)),
 ('For', (691, 1))]

In [ ]:
# Group words for words.txt according to their first 4 characters and then output the number of members for the first 10 groups.

In [ ]:
lines4 = sc.textFile("words.txt")
test1_3 = lines3.zipWithIndex().flatMap(lambda x: ((i,(x[1],1)) for i in x[0].split(" ")))
test2_3 = test1_3.map(lambda x: ((x[0], x[1][0]), x[1][1])).reduceByKey(lambda x, y: x + y)
Result_RDD3 = test2_3.map(lambda x: (x[0][0], (x[0][1], x[1])))